In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import sys
import time
from os.path import join, normpath
import roamability as rb

In [1]:
from pathlib import Path
Path.cwd()

WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

## 1. Отчет по странам с небольшим или отрицательным профитом

In [1]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

In [3]:
sql_srt=\
'''
--Calculate total charge and cost by MCC, MNC, usage_type, account_id
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,u.mnc,u.usage_type,uc.account_id
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost from [10028_usage2018-04].[dbo].[USAGE] u
inner join [10028_usage2018-04].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,u.mnc,u.usage_type,uc.account_id
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

In [4]:
df = pd.read_sql_query(sql_srt, cnxn,parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

,REP_DATE,mcc,mnc,usage_type,account_id,quantity,charge,cost
0,2018-04-01,202,5,15,352321.0,4.05000,0.00000,0.00000
1,2018-04-01,202,5,15,352370.0,278.61663,0.00000,0.00000
2,2018-04-01,202,5,21,352321.0,18.00000,0.00000,0.00000
3,2018-04-01,202,5,21,352334.0,26.00000,0.00000,0.00000
4,2018-04-01,202,5,22,352334.0,63.00000,0.63945,0.37044


In [5]:
def f(x):
    data=x[['charge','cost']]
    x['profit']=data['charge']-data['cost']
    if data['cost']!=0:
        x['profit_per']=round((x['profit']/data['cost'])*100,1)
    else:
        x['profit_per']=0
    return x

df1=df.groupby(['mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()

In [6]:
df1[df1['profit']<0]

charge         cost      profit  profit_per
mcc mnc                                                 
208 10      2.37834     34.06715   -31.68881       -93.0
    29    233.39442    274.69781   -41.30339       -15.0
214 9      24.09512    676.18343  -652.08831       -96.4
234 3       0.27545      0.91020    -0.63475       -69.7
240 10      0.07635      0.14915    -0.07280       -48.8
310 260    10.61740    677.87273  -667.25533       -98.4
    680  1465.88595   2125.99344  -660.10749       -31.0
515 2    6012.19448  10894.22103 -4882.02655       -44.8
901 18      0.74520      8.12001    -7.37481       -90.8

In [7]:
df2=df.groupby(['REP_DATE','mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()
df2.head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-02-16,530,1,0.00026,0.00015,0.00011,73.3
2018-02-21,222,10,0.78593,0.45280,0.33313,73.6
2018-02-22,222,10,0.28601,0.16414,0.12187,74.2
2018-02-23,222,10,0.05382,0.02994,0.02388,79.8
2018-02-24,222,10,0.09339,0.05312,0.04027,75.8


In [9]:
df2[(df2.index.get_level_values('mcc')==310) & (df2.index.get_level_values('mnc')==680)].head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-04-01,310,680,78.21434,72.55151,5.66283,7.8
2018-04-02,310,680,42.00338,53.25511,-11.25173,-21.1
2018-04-03,310,680,64.79633,61.03443,3.76190,6.2
2018-04-04,310,680,173.01345,144.49367,28.51978,19.7
2018-04-05,310,680,29.24078,47.55932,-18.31854,-38.5


## 2. Сравнительный анализ цен в TAP и Cost в OCS

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import mysql.connector #pip install mysql-connector-python-rf
from os.path import join, normpath

In [2]:
# Задать переменные и загрузить вспомогательные данные

data_tariff_allowed = 0.1 # Тариф для data в USD, который рассматривается как discounter
tap_data_tot_charge_threshold = 10 # Полная цена по PLMN, ниже которой не анализируется разница в процентах между тарифами ТАР и OCS
tap_ocs_tariff_diff = 50 # Допустимая разница в процентах между тарифами ТАР и OCS

downloads = 'C:/Users/balob/Downloads'

df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Sun_Dec_29_2019.csv'))

### Получение данных

In [3]:
#Из базы данных BSS выгружаем данные по тарифам

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig AS TADIG
,CostTariffId AS Price_Plan
,MIN((Cost*1024*1024)/TotalChargedUnits) AS tap_data_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS tap_data_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS tap_data_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS tap_data_max
,SUM(Cost) AS tap_data_tot_charge
,SUM(Revenue) AS tap_data_tot_revenue
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20191200000000
AND TotalChargedUnits > 1024
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_tap_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap_tariff.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue
0,AIACW,Partner Cost Plan 2018,0.018398,0.024574,0.002740,0.025938,1.603231,3.206414
1,ALBEM,P4 Cost Plan 2018,0.042247,0.042991,0.000397,0.043811,10.450860,22.682369
2,ALBVF,Partner Cost Plan 2018,0.000000,0.001794,0.001735,0.006105,34.039169,68.106679
3,AREDU,MB Cost Plan 2019,0.000000,0.008056,0.005942,0.020118,264.625825,524.263598
4,ARGTM,Partner Cost Plan 2018,0.000000,0.017098,0.007453,0.046880,1021.307002,2042.414515


In [6]:
#Из базы данных OCS выгружаем тарифы и объединяем с данными BSS и сохраняем в файл csv

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT t.tadig_code AS TADIG,p.roaming_plan_name  AS Price_Plan
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_call_rate WHEN rpr.currency_id=2 THEN rpr.mo_call_rate*1.2 END ocs_moc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_call_rate WHEN rpr.currency_id=2 THEN rpr.mt_call_rate*1.2 END ocs_mtc_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_sms_rate WHEN rpr.currency_id=2 THEN rpr.mo_sms_rate*1.2 END ocs_mo_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_sms_rate WHEN rpr.currency_id=2 THEN rpr.mt_sms_rate*1.2 END ocs_mt_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.data_rate WHEN rpr.currency_id=2 THEN rpr.data_rate*1.2 END ocs_data_rate
,rpr.is_discounted
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_TADIG_CODES t
ON rpr.network_id=t.network_id
LEFT JOIN ROAMING_PLANS p
ON rpr.roaming_plan_id=p.roaming_plan_id
WHERE rpr.roaming_plan_id IN (267,268,329,343,372)
AND ((rpr.start_date < GETDATE() AND rpr.end_date is null) OR (rpr.start_date < GETDATE() AND rpr.end_date > GETDATE()))
AND p.deleted IS NULL
'''

df_ocs_tariff = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

df = pd.merge(df_tap_tariff,df_ocs_tariff[['TADIG','Price_Plan','ocs_data_rate']],
        how='left',on=['TADIG','Price_Plan'])

df['pct_diff'] = (df['tap_data_avg'] - df['ocs_data_rate'])*100 / df['ocs_data_rate']

#df.to_csv('./DATA/tap_ocs_analysis_180817.csv',index=False)
df.head()

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
0,AIACW,Partner Cost Plan 2018,0.018398,0.024574,0.002740,0.025938,1.603231,3.206414,0.0257,-4.379824
1,ALBEM,P4 Cost Plan 2018,0.042247,0.042991,0.000397,0.043811,10.450860,22.682369,0.0468,-8.139600
2,ALBVF,Partner Cost Plan 2018,0.000000,0.001794,0.001735,0.006105,34.039169,68.106679,0.0030,-40.192740
3,AREDU,MB Cost Plan 2019,0.000000,0.008056,0.005942,0.020118,264.625825,524.263598,0.0120,-32.864562
4,ARGTM,Partner Cost Plan 2018,0.000000,0.017098,0.007453,0.046880,1021.307002,2042.414515,0.0180,-5.008522


### Анализ данных

In [13]:
#Для анализа можно загрузить уже имеющиеся данные

#df = pd.read_csv('./DATA/tap_ocs_analysis_180817.csv')

In [7]:
# Находим тарифы TAP выше заданного допустимого уровня для данного типа тарифа

df[df.tap_data_avg > data_tariff_allowed].sort_values(by=['tap_data_avg'], ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
331,VENMS,Partner Cost Plan 2018,0.200058,0.203589,0.016394,0.280549,3.736748,7.466256,0.20,1.794498
264,PHLSR,Partner Cost Plan 2018,0.000000,0.151902,0.038011,0.317277,83.063284,166.108602,0.15,1.267887
152,INDBI,Partner Cost Plan 2018,0.019717,0.113068,0.050053,0.196959,9.813200,6.774665,0.02,465.341934


In [8]:
# Находим тарифы с наибольшей разницей между данными ТАР и тарифами OCS

df[(df.tap_data_tot_charge > tap_data_tot_charge_threshold)&(df.pct_diff > tap_ocs_tariff_diff)]\
.sort_values(by=['pct_diff'],ascending=False)

,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
137,IND07,Partner Cost Plan 2018,0.019715,0.045039,0.030717,0.196959,10.529570,14.206132,0.02,125.197250
156,INDEU,Partner Cost Plan 2018,0.019715,0.044866,0.031172,0.196959,34.708703,48.087688,0.02,124.331459
161,INDIM,Partner Cost Plan 2018,0.019715,0.042920,0.018967,0.196959,51.547776,66.261067,0.02,114.599221
167,INDMT,Partner Cost Plan 2018,0.009858,0.018964,0.013740,0.098480,14.023764,25.230555,0.01,89.640459
166,INDMP,Partner Cost Plan 2018,0.019715,0.037872,0.020109,0.196959,29.285741,43.019673,0.02,89.359829
148,INDA7,Partner Cost Plan 2018,0.009858,0.018487,0.015737,0.098480,37.877895,74.206598,0.01,84.874823
157,INDH1,Partner Cost Plan 2018,0.009858,0.018114,0.014619,0.098548,125.367990,232.864127,0.01,81.137718
149,INDA8,Partner Cost Plan 2018,0.009858,0.017478,0.011862,0.098480,83.064744,161.893448,0.01,74.783050
142,INDA1,Partner Cost Plan 2018,0.009858,0.017282,0.008553,0.098480,433.085061,776.156849,0.01,72.819120
165,INDJH,Partner Cost Plan 2018,0.009858,0.017006,0.013674,0.098548,146.644923,286.766152,0.01,70.057720


In [10]:
# Анализ данных по Allowed list в DMI и тарифов в OCS

tadig = 'INDIM'

display(df_ocs_tariff[df_ocs_tariff.TADIG == tadig])
display(df[df.TADIG == tadig])
display(df_al[(df_al['PLMN code'] == tadig) & (df_al['Sponsor'].str.contains('', na=False))])

,TADIG,Price_Plan,ocs_moc_rate,ocs_mtc_rate,ocs_mo_sms_rate,ocs_mt_sms_rate,ocs_data_rate,is_discounted
510,INDIM,Partner Cost Plan 2018,0.1,0.0,0.06,0.0,0.02,True


,TADIG,Price_Plan,tap_data_min,tap_data_avg,tap_data_std,tap_data_max,tap_data_tot_charge,tap_data_tot_revenue,ocs_data_rate,pct_diff
145,INDIM,Partner Cost Plan 2018,0.01961,0.04519,0.017896,0.196903,12.985733,15.3538,0.02,125.947616


,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb
1863,Partner_Clay_Com,INDIM,1,1,1,1,1,1,NaN,NaN
3334,Partner_Combined,INDIM,1,1,1,1,1,1,NaN,NaN
3484,Partner_RedT_Sof,INDIM,1,1,1,1,1,1,NaN,NaN
3498,Partner_RedTea,INDIM,1,1,1,1,1,1,NaN,NaN
4009,Partner_RUSEC_Co,INDIM,1,1,1,1,1,1,NaN,NaN
4146,Partner_Com_JWM,INDIM,1,1,1,1,1,1,NaN,NaN
5272,Partner_All,INDIM,1,1,1,1,1,1,NaN,NaN


In [10]:
df_al[(df_al['PLMN code'].str.startswith('BELKO', na=False)) & (df_al['Sponsor'].str.contains('P4', na=False))] 

,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb
266,P4_STI_Comb,BELKO,1,1,1,1,1,1,NaN,NaN
469,P4_tinklabs,BELKO,1,1,1,1,1,1,NaN,NaN
584,P4_mon-sky,BELKO,1,1,0,1,1,1,NaN,NaN
643,P4_2018,BELKO,1,1,1,1,1,1,NaN,NaN
752,P4_WeStream,BELKO,1,1,0,1,1,1,NaN,NaN
884,P4_All Rated,BELKO,1,1,0,1,1,1,NaN,NaN
1006,P4_mon_bss,BELKO,1,1,1,1,1,1,NaN,NaN
1350,P4_mon_euro,BELKO,1,1,1,1,1,1,NaN,NaN
1530,P4_PS,BELKO,1,1,1,1,1,1,NaN,NaN
1531,P4_telzar,BELKO,1,1,1,1,1,1,NaN,NaN


In [8]:
#Анализ данных из BSS по заданному тарифу

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
DATE_FORMAT(Search_utcTimeStamp, "%Y-%m-%d %H:%i") AS RED_DATE
,VisitedNetworkTadig
,Cost
,CostTariffId
,TotalChargedUnits
,(Cost*1024*1024)/TotalChargedUnits AS Price
,GprsDest_accessPointNameNI
,InTapFileName
,OutTapFileName
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20190814000000
AND VisitedNetworkTadig = 'MYSMI' AND Cost > 0
AND CostTariffId LIKE 'MB%'
AND TotalChargedUnits > 1024000
LIMIT 300
'''

df_tap = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()
df_tap.take([0,1,2,4,5,6,7],axis=1).head()

,RED_DATE,VisitedNetworkTadig,Cost,TotalChargedUnits,Price,GprsDest_accessPointNameNI,InTapFileName
0,2019-04-09 02:34,MYSMI,0.494050,12738560,0.040668,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00021_RB
1,2019-04-09 04:30,MYSMI,0.052884,1402880,0.039528,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00021_RB
2,2019-04-09 04:48,MYSMI,0.057059,1464320,0.040859,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00021_RB
3,2019-04-09 04:51,MYSMI,0.346531,8919040,0.040740,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00021_RB
4,2019-04-09 05:27,MYSMI,0.038967,1075200,0.038002,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00021_RB


In [9]:
df_tap.take([0,1,2,4,5,6,7],axis=1).tail()

,RED_DATE,VisitedNetworkTadig,Cost,TotalChargedUnits,Price,GprsDest_accessPointNameNI,InTapFileName
83,2019-04-07 10:49,MYSMI,0.038967,1054720,0.038740,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00019_RB
84,2019-04-07 11:18,MYSMI,0.061234,1648640,0.038947,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00019_RB
85,2019-04-07 10:57,MYSMI,0.137777,3563520,0.040541,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00019_RB
86,2019-04-07 12:26,MYSMI,0.119685,3092480,0.040582,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00019_RB
87,2019-04-07 11:46,MYSMI,0.961658,24739840,0.040759,mobile.three.com.hk,mbsponsortap/incoming/CDMYSMIMVNMB00019_RB


### Анализ разброса cost в TAP по сравнению с OCS cost

In [ ]:
# Загрузить данные из BSS

sql_srt='''
SELECT
SponsorPrefix
,Cost
,CostTariffId
,VisitedNetworkId
,TotalChargedUnits
FROM TAP.GPRS_CALL
WHERE TotalChargedUnits > 0
AND TreatedTimestampUtc >= 20190901000000 AND TreatedTimestampUtc < 20191001000000
'''
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_gprs = pd.read_sql_query(sql_srt, cnxn)

df_bss_gprs = pd.read_csv(join(downloads, 'df_bss_gprs.csv'))
df_bss_gprs.SponsorPrefix.replace({42501:'1',26006:'2',45403:'4'}, inplace=True)
df_bss_gprs.SponsorPrefix = df_bss_gprs.SponsorPrefix.astype('int64')
df_bss_gprs.head(3)
df_bss_gprs.head()

In [ ]:
# Загрузить тарифы из OCS

sql_srt=\
'''
SELECT r.NETWORK_ID AS VisitedNetworkId
,r.SPONSOR_ID AS SponsorPrefix
,r.data_rate AS OcsDataCost
FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329,343)
AND r.end_date IS NULL
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_disc = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_disc.VisitedNetworkId = df_disc.VisitedNetworkId.astype('int64')
df_disc.SponsorPrefix = df_disc.SponsorPrefix.astype('int64')
df_disc.head(3)

In [ ]:
# Объединить тарифы и данные BSS

df_temp = pd.merge(df_bss_gprs[df_bss_gprs.Cost > 0.2],
                   df_disc, how='left', on=['VisitedNetworkId','SponsorPrefix'])
df_temp[['TapDataCost', 'PctDif']] = df_temp.apply(lambda x: Series([
    x['Cost'] * 1024 * 1024 / x['TotalChargedUnits'],
    ((x['Cost'] * 1024 * 1024 / x['TotalChargedUnits'] - x['OcsDataCost'])*100)/x['OcsDataCost']
                               ]), axis=1)

df_temp.head()

In [ ]:
# Построить график разброса

df_temp.PctDif.plot.hist(bins=50, edgecolor='black', color='green', alpha=0.5, density=True, log=True)
ax = plt.gca() # get the current axes
ax.set_xlabel('Difference, %')
plt.show()

# Тестирование

#### Парсинг TAP файлов

In [ ]:
from pandas import DataFrame
from bs4 import BeautifulSoup
tap_file = open(join(downloads,'CPVTM.txt')).read()
#tap_file = open(join(downloads,'TAP.txt')).read()
soup = BeautifulSoup(tap_file, 'xml')
print(soup.find('sender').text,soup.find('localCurrency').text,soup.find('exchangeRate').text)
for callEventDetails in soup.find_all('callEventDetails'):
    print(callEventDetails.imsi.text,
          callEventDetails.charge.text,
          callEventDetails.chargeableUnits.text)